1. **Filter columns**: only get the ones I need (keep NaN for now, they won't be counted anyway)
2. **Filter date** using *pd.to_datetime('Series').dt.[year/month]*: only keep the ones happened in 2016
3. **Add columns** 'MONTH', 'MANHATTAN' and rename column 'BOROUGH' to 'NYC'
4. **Group by 'MONTH'**, get the count Series for 'NYC', do sum to 'MANHATTAN' because it's a bool Series
5. **Put two Series** back in dataframe, add 'PERCENTAGE' column

### Step 1

In [1]:
from pandas import Series, DataFrame
import pandas as pd
import os

In [2]:
df_origin = pd.read_csv(os.getcwd() + '/Data/vehicle_collisions.csv')
df_origin.head(5)

UNIQUE KEY    DATE  TIME   BOROUGH  ZIP CODE   LATITUDE  LONGITUDE  \
0     3146911  1/1/15  0:20    QUEENS   11358.0  40.751847 -73.787862   
1     3146180  1/1/15  0:20       NaN       NaN  40.771289 -73.946693   
2     3146384  1/1/15  0:21  BROOKLYN   11205.0  40.689445 -73.955121   
3     3146013  1/1/15  0:30  BROOKLYN   11213.0  40.673845 -73.925080   
4     3146120  1/1/15  0:33       NaN       NaN        NaN        NaN   

                    LOCATION    ON STREET NAME   CROSS STREET NAME  \
0   (40.7518471, -73.787862)         47 AVENUE          193 STREET   
1  (40.7712888, -73.9466928)               NaN                 NaN   
2  (40.6894449, -73.9551212)    BEDFORD AVENUE    LAFAYETTE AVENUE   
3  (40.6738445, -73.9250801)    BUFFALO AVENUE  SAINT MARKS AVENUE   
4                        NaN  RICHMOND TERRACE        SOUTH AVENUE   

        ...                      VEHICLE 1 TYPE     VEHICLE 2 TYPE  \
0       ...         SPORT UTILITY/STATION WAGON                NaN   
1       ...                   PASSENGER VEHICLE                NaN   
2       ...                   PASSENGER VEHICLE            UNKNOWN   
3       ...                                 BUS  PASSENGER VEHICLE   
4       ...                             UNKNOWN  PASSENGER VEHICLE   

   VEHICLE 3 TYPE  VEHICLE 4 TYPE  VEHICLE 5 TYPE  \
0             NaN             NaN             NaN   
1             NaN             NaN             NaN   
2             NaN             NaN             NaN   
3             NaN             NaN             NaN   
4             NaN             NaN             NaN   

              VEHICLE 1 FACTOR  VEHICLE 2 FACTOR  VEHICLE 3 FACTOR  \
0  TRAFFIC CONTROL DISREGARDED               NaN               NaN   
1               ANIMALS ACTION               NaN               NaN   
2              FATIGUED/DROWSY       UNSPECIFIED               NaN   
3           LOST CONSCIOUSNESS               NaN               NaN   
4                  UNSPECIFIED       UNSPECIFIED               NaN   

   VEHICLE 4 FACTOR VEHICLE 5 FACTOR  
0               NaN              NaN  
1               NaN              NaN  
2               NaN              NaN  
3               NaN              NaN  
4               NaN              NaN  

[5 rows x 29 columns]

In [3]:
df_step1 = df_origin[['DATE','BOROUGH']]
df_step1.head(5)

DATE   BOROUGH
0  1/1/15    QUEENS
1  1/1/15       NaN
2  1/1/15  BROOKLYN
3  1/1/15  BROOKLYN
4  1/1/15       NaN

### Step 2

In [4]:
df_step2 = df_step1[pd.to_datetime(df_step1['DATE']).dt.year==2016]
df_step2.head(5)

DATE    BOROUGH
217539  1/1/16  MANHATTAN
217540  1/1/16   BROOKLYN
217541  1/1/16  MANHATTAN
217542  1/1/16  MANHATTAN
217543  1/1/16  MANHATTAN

In [6]:
type(pd.to_datetime(df_step2['DATE']))

pandas.core.series.Series

### Step 3
I found this method Series.dt.strftime('%b'), this follows same rule as Datetime module

In [7]:
df_step3 = df_step2.copy()

In [11]:
df_step3.insert(loc=1, column='MONTH', value=pd.to_datetime(df_step3['DATE']).dt.strftime('%b'))

In [12]:
df_step3.head(5)

DATE MONTH    BOROUGH
217539  1/1/16   Jan  MANHATTAN
217540  1/1/16   Jan   BROOKLYN
217541  1/1/16   Jan  MANHATTAN
217542  1/1/16   Jan  MANHATTAN
217543  1/1/16   Jan  MANHATTAN

In [13]:
df_step3 = df_step3.drop('DATE', axis=1)
df_step3.head(5)

MONTH    BOROUGH
217539   Jan  MANHATTAN
217540   Jan   BROOKLYN
217541   Jan  MANHATTAN
217542   Jan  MANHATTAN
217543   Jan  MANHATTAN

In [14]:
df_step3['MANHATTAN'] = df_step3['BOROUGH'].values==['MANHATTAN']
df_step3.head(5)

MONTH    BOROUGH MANHATTAN
217539   Jan  MANHATTAN      True
217540   Jan   BROOKLYN     False
217541   Jan  MANHATTAN      True
217542   Jan  MANHATTAN      True
217543   Jan  MANHATTAN      True

In [15]:
df_step3 = df_step3.rename(columns={'BOROUGH':'NYC'})
df_step3.head(5)

MONTH        NYC MANHATTAN
217539   Jan  MANHATTAN      True
217540   Jan   BROOKLYN     False
217541   Jan  MANHATTAN      True
217542   Jan  MANHATTAN      True
217543   Jan  MANHATTAN      True

In [16]:
df_step3['NYC'] = df_step3['NYC'].fillna(value="NOT IN RECORD")
df_step3.head(5)

MONTH        NYC MANHATTAN
217539   Jan  MANHATTAN      True
217540   Jan   BROOKLYN     False
217541   Jan  MANHATTAN      True
217542   Jan  MANHATTAN      True
217543   Jan  MANHATTAN      True

### Step 4

In [17]:
series_nyc = df_step3.groupby('MONTH')['NYC'].count()

In [18]:
series_nyc.head(3)

MONTH
Apr    18331
Aug    19678
Dec    19214
Name: NYC, dtype: int64

In [19]:
series_manhattan = df_step3.groupby('MONTH')['MANHATTAN'].sum()

In [20]:
series_manhattan.head(3)

MONTH
Apr    3540.0
Aug    2844.0
Dec    2634.0
Name: MANHATTAN, dtype: float64

### Part 5

In [21]:
df_output = DataFrame(series_manhattan, columns=['MANHATTAN'])
df_output['NYC'] = series_nyc
df_output['MANHATTAN'] = df_output['MANHATTAN'].astype(int)
df_output.head(5)

MANHATTAN    NYC
MONTH                  
Apr         3540  18331
Aug         2844  19678
Dec         2634  19214
Feb         3195  15985
Jan         3178  18101

In [22]:
df_output['PERCENTAGE'] = df_output['MANHATTAN']/df_output['NYC']

In [23]:
df_output.head(5)

MANHATTAN    NYC  PERCENTAGE
MONTH                              
Apr         3540  18331    0.193115
Aug         2844  19678    0.144527
Dec         2634  19214    0.137088
Feb         3195  15985    0.199875
Jan         3178  18101    0.175570

In [24]:
df_output.to_csv(os.getcwd()+ '/Q1_Part1_output.csv')

Since I've already finished, I don't want to go back fix the month order...the way to fix is DO NOT remove number month column....